In [ ]:
# =======================================================================
# === CELL 1: INSTALLS ===
# =======================================================================
# This block installs all the necessary Python libraries.
# - simplegmail: For accessing Gmail
# - txtai[pipeline]: For the summarization and tone models
# - spacy: For to-do and deadline (Named Entity Recognition) extraction
print("Installing libraries...")
!pip install --upgrade google-api-python-client oauth2client simplegmail
!pip -q install txtai[pipeline] > /dev/null
!pip install spacy
!python -m spacy download en_core_web_sm
print("Installations complete.")


In [ ]:
# =======================================================================
# === CELL 2: IMPORTS ===
# =======================================================================
# Here we import all the functions and classes we'll need from the libraries.
from google.colab import files
from simplegmail import Gmail
from simplegmail.query import construct_query
from txtai.pipeline import Summary, TextClassifier
import spacy
import re


In [ ]:
# =======================================================================
# === CELL 3: GMAIL AUTHENTICATION ===
# =======================================================================
# This section handles logging into your Google account.
# It will ask you to upload your 'client_secret.json' file.
# Then, it will open a browser link for you to approve access.
print("Please upload your 'client_secret.json' file.")
uploaded = files.upload()

print("Initializing Gmail... Please follow the authentication link.")
gmail = Gmail()
print("Gmail authentication successful.")


In [ ]:
# =======================================================================
# === CELL 4: GET USER INPUT FOR QUERY ===
# =======================================================================
# This function (get_query_params) runs interactive prompts
# to ask you what emails to search for (e.g., unread, from sender, etc.).
def get_query_params():
    """Interactively prompts user for email search criteria."""
    query_params = {}

    # Get 'read' status
    while True:
        read_input = input("Enter 'read' status (True/False) or press Enter to skip: ")
        if read_input == '':
            break
        elif read_input.lower() in ['true', 'false']:
            query_params["read"] = read_input.lower() == 'true'
            break
        else:
            print("Invalid input. Please enter 'True' or 'False'.")

    # Get 'sender'
    while True:
        sender_input = input("Enter sender email(s) (comma-separated) or press Enter to skip: ")
        if sender_input == '':
            break
        elif all('@' in email.strip() for email in sender_input.split(',')):
            query_params["sender"] = [email.strip() for email in sender_input.split(',')]
            break
        else:
            print("Invalid input. Please enter valid email addresses.")

    # Get 'newer_than'
    while True:
        newer_than_input = input("Enter 'newer_than' duration (e.g., '5 day') or press Enter to skip: ")
        if newer_than_input == '':
            break
        try:
            number, unit = newer_than_input.split()
            query_params["newer_than"] = (int(number), unit)
            break
        except ValueError:
            print("Invalid input for 'newer_than'. It should be in the format: '<number> <unit>'")

    # Get 'labels'
    while True:
        labels_input = input("Enter labels (comma-separated, e.g., 'CATEGORY_FORUMS, INBOX') or press Enter to skip: ")
        if labels_input == '':
            break
        elif all(label.strip() for label in labels_input.split(',')):
            query_params["labels"] = [[label.strip()] for label in labels_input.split(',')]
            break
        else:
            print("Invalid input. Please enter valid labels.")

    return query_params

# Run the function to get the search parameters
print("\n--- Email Search Query ---")
query_params = get_query_params()
print("\nGenerated query_params:")
print(query_params)


In [ ]:
# =======================================================================
# === CELL 5: FETCH GMAIL MESSAGES ===
# =======================================================================
# This block uses the query from Cell 4 to search your Gmail
# and retrieve the matching email messages.
print("\nFetching emails from Gmail...")
promo_query = construct_query(query_params)
output = gmail.get_messages(query=promo_query)
print(f"Found {len(output)} emails.")


In [ ]:
# =======================================================================
# === CELL 6: INITIALIZE AI MODELS ===
# =======================================================================
# This loads the three pre-trained AI models into memory.
# 1. Summary: For summarizing email text.
# 2. TextClassifier: For detecting tone (sentiment).
# 3. spacy (nlp): For extracting dates, to-dos, etc. (Named Entity Recognition).
#
print("Loading AI models (this may take a moment)...")
summary = Summary()
tone_classifier = TextClassifier("cardiffnlp/twitter-roberta-base-sentiment")
nlp = spacy.load("en_core_web_sm")
print("All models loaded.")


# =======================================================================
# === CELL 6.5: HELPER FUNCTIONS ===
# =======================================================================
# These are the custom functions for analysis.

def score_urgency(text):
    """Scores urgency based on keywords. Returns a numeric score."""
    text_lower = text.lower()
    score = 0
    if "urgent" in text_lower or "asap" in text_lower:
        score += 10
    if "important" in text_lower or "action required" in text_lower:
        score += 5
    if "deadline" in text_lower or "please review" in text_lower:
        score += 3
    if "!" in text:
        score += 1
    return score

def extract_actions(text, nlp_model):
    """Uses spaCy to find potential to-dos and deadlines."""
    doc = nlp_model(text)
    deadlines = []
    todos = []

    # Find all 'DATE' entities
    for ent in doc.ents:
        if ent.label_ == "DATE":
            deadlines.append(ent.text.strip())

    # Find potential action verbs (simplified)
    for token in doc:
        if token.pos_ == "VERB" and token.dep_ == "ROOT":
            action_phrase = " ".join([t.text for t in token.subtree])
            todos.append(action_phrase.strip())

    # Find lines explicitly asking for action
    for line in text.split('\n'):
        if re.search(r"(to-do|action item):", line, re.IGNORECASE):
            todos.append(line)

    return list(set(todos)), list(set(deadlines))


In [ ]:
# =======================================================================
# === CELL 7: PROCESS EMAILS & BUILD SUMMARY ===
# =======================================================================
# This is the main part of the script.
# 1. It loops through each email.
# 2. It analyzes each one for summary, urgency, tone, and actions.
# 3. It stores the results in a list.
# 4. It sorts the list by the urgency score.
# 5. It builds the final mail_contents string.
# 6. It saves the results to 'mail_collector.txt'.

processed_emails = []
print(f"\nProcessing and analyzing {len(output)} emails...")

# --- STEP 1: Process and Analyze all emails ---
for i, mail in enumerate(output):
    # This print statement will overwrite itself, creating a nice progress bar
    print(f"  Analyzing email {i+1} of {len(output)}...", end='\r')
    if not mail.plain:
        continue # Skip emails with no plain text

    text_content = mail.plain

    # Run the 4 analysis functions
    summary_text = summary(text_content, minlength=20, maxlength=75)
    urgency_score = score_urgency(mail.subject + " " + text_content)
    tone_result = tone_classifier(summary_text)
    tone = tone_result[0]['label']
    todos, deadlines = extract_actions(text_content, nlp)

    # Store everything
    processed_emails.append({
        "sender": mail.sender,
        "subject": mail.subject,
        "date": mail.date,
        "summary": summary_text,
        "urgency_score": urgency_score,
        "tone": tone.capitalize(),
        "todos": todos,
        "deadlines": deadlines
    })

# --- STEP 2: Prioritize (Sort) the emails ---
processed_emails.sort(key=lambda x: x['urgency_score'], reverse=True)
print("\nEmail analysis and prioritization complete.     ") # Spaces to clear the progress bar

# --- STEP 3: Build the final output string ---
mail_contents = "Your Prioritized Email Summary:\n\n"

for email_data in processed_emails:
    urgency_display = f" (URGENCY: {email_data['urgency_score']})" if email_data['urgency_score'] > 0 else ""

    mail_contents += "========================================\n"
    mail_contents += f"From: {email_data['sender']}\n"
    mail_contents += f"Subject: {email_data['subject']}{urgency_display}\n"
    mail_contents += f"Date: {email_data['date']}\n"
    mail_contents += f"Tone: {email_data['tone']}\n"

    mail_contents += f"\nSummary:\n{email_data['summary']}\n"

    if email_data['todos']:
        mail_contents += "\nPotential To-Dos:\n"
        for todo in email_data['todos']:
            mail_contents += f"  - {todo}\n"

    if email_data['deadlines']:
        mail_contents += "\nPotential Deadlines:\n"
        for deadline in email_data['deadlines']:
            mail_contents += f"  - {deadline}\n"

    mail_contents += "\n\n"

# --- STEP 4: Save to file ---
# Using 'w' (write) to overwrite the file each time you run the script
with open('mail_collector.txt', 'w') as col:
    col.write(mail_contents)
print('Write completed to mail_collector.txt')


In [ ]:
# =======================================================================
# === CELL 8: SEND THE FINAL EMAIL ===
# =======================================================================
# This block takes the final mail_contents string and
# sends it as an email from you, to you (or to whomever you specify).

# --- IMPORTANT: Change these email addresses ---
email = "poojapriya2705@gmail.com"        # The address to SEND TO
sender_email = "poojapriya2705@gmail.com" # The address to SEND FROM (must be you)
# ------------------------------------------------

params={
    "to": email,
    "sender": sender_email,
    "subject": "Your Prioritized Email Summary",
    "msg_plain": mail_contents
}

print(f"Sending summary email to {email}...")
message = gmail.send_message(**params)

print('Email sent successfully!')
print('--- All tasks complete. ---')